In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import time
from sapy.components import Clock, ProgramCounter, MemoryAddressRegister, \
    RandomAccessMemory, SwitchBoard, RegisterA, RegisterB, \
    RegisterOutput, ArithmeticUnit, RegisterInstruction, Computer, opcode_map
from sapy.assembler import assemble

# Hex and Binary Literals in Python

In [ ]:
0xF

In [ ]:
0xFF

In [ ]:
print(15_000) # underscore as separators are allowed
print(0b1111_1111)

You can also format integers to HEX using X

In [ ]:
f"0x{232:02X}"

# Assembly Language

In [ ]:
assemble("LDA #$07")

# Clocking Signals

In [ ]:
clk = Clock()
class DummyComponent():
    def data(self, con=tuple()):
        print("data accessed")
        return 42
    def clock(self, data, con=tuple()):
        print("clock stepped")

dc = DummyComponent()
clk.add_component(dc)

In [ ]:
print("step 1: ---------")
clk.step()
print("step 2: ---------")
clk.step()

# Program Counter

To enable a component to take an action on a clock, you pass in the correct control bit as a keyword argument. To enable the PC to output to the data but, use the "ep" control bit.

In [ ]:
pc = ProgramCounter()

In [ ]:
pc.clock()
pc.data()

In [ ]:
pc.clock()
pc.data(con=['ep'])

In [ ]:
pc.clock()
pc.data(con=['ep'])

To increment the program counter, the control bit is "cp".

In [ ]:
pc.clock(con=['cp'])
pc.data(con=['ep'])

In [ ]:
pc.clock(con=['cp'])
pc.data(con=['ep'])

In [ ]:
pc.clock(con=['cp'])
pc.data(con=['ep'])

For jump instructions you set the program counter directly. This is done with the "lp" control bit.

In [ ]:
pc.clock(data=0xD, con=['lp'])
print(f"0x{pc.data(con=['ep']):02X}")
print(f"{pc.data(con=['ep'])}")

# Accumulator (Register A)

In [ ]:
reg_a = RegisterA()
print(f"The register value on reset is: 0x{reg_a.data(con=['ea']):0X}")

reg_a.clock(data=0xAB, con=['la'])
reg_a.data(con=['ea'])
print(f"The register has latched a value: 0x{reg_a.data(con=['ea']):0X}")


# ALU
## Arithmatic Logic Unit

Some components have extra attributes that allow it to communicate directly to other components outside of the bus. 

The ALU is not clocked, it doesn't even have state. The data output from the ALU is alway the instantaneous sum (or difference) of Reg A + Reg B

In [ ]:
reg_a = RegisterA()
reg_a.clock(data=4, con=['la'])

reg_b = RegisterB()
reg_b.clock(data=3, con=['lb'])

alu = ArithmeticUnit(reg_a, reg_b)

alu.data(con=['eu'])

print(f"\t0x{reg_a.value:02X}")
print(f"+\t0x{reg_b.value:02X}")
print('-' * 12)
print(f"\t0x{alu.data(con=['eu']):02X}")

In [ ]:
# change the value of the register feeding the alu
reg_b.clock(data=1, con=['lb'])
alu.data(con=['eu'])

print(f"\t0x{reg_a.value:02X}")
print(f"+\t0x{reg_b.value:02X}")
print('-' * 12)
print(f"\t0x{alu.data(con=['eu']):02X}")

Subtraction

In [ ]:

print(f"\t0x{reg_a.value:02X}")
print(f"-\t0x{reg_b.value:02X}")
print('-' * 12)
print(f"\t0x{alu.data(con=['eu', 'su']):02X}")

# Memory

Ram is another example of a system that has communication outside the bus, it always access the RAM value stored at the address latched in the Memory Address Register.

In [ ]:
mar = MemoryAddressRegister()
ram = RandomAccessMemory(mar)

# store address for ram in register
mar.clock(data=0x0F, con=['lm'])
print(f"Memory Address: 0x{mar.value:02X}")


# clock data into ram at the address set above
ram.clock(data=0xAB, con=['lr'])
assert ram.data() == None
print(f"Memory at address: 0x{mar.value:02X} is 0x{ram.data(con=['er']):02X}") # should be 0xAB
print(f"Memory at address: {mar.value} is {ram.data(con=['er'])}") # just to demystify hex..

# Controller: Sequencer + Decoder + Clock

In [ ]:
reg_i = RegisterInstruction()
instruction = 0xFF # both opcode and argument, 8bits
reg_i.clock(data=instruction, con=['li'])


In [ ]:
print(f"The opcode is {reg_i.value:X}")

# Example Sequencing: LDA

In [ ]:
from sapy.components import Computer

cpu = Computer()
rom = [
    0x00, 0x09, # 0x0 LDA 09H
    ]
cpu.switches.load_program(rom)

for _ in range(7):
    cpu.step(debug=True) 

In [ ]:
cpu.step(debug=True)

# Output (Register O)

In [ ]:
reg_o = RegisterOutput()
reg_o.clock(data=0x90, con=['lo'])

In [ ]:
# monkey patch output function for fancy display
def display_number(x):
    print(f"***### {x} ###***")
reg_o.output_function = display_number

In [ ]:
reg_o.clock(data=0x90, con=['lo'])

# Full Computer with ROM

In [ ]:
cpu = Computer()
rom = [
    0x00, 0x07, # 0x00 LDA 07H
    0x01, 0x08, # 0x02 ADD 08H
    0xF6,       # 0x04 OTA
    0x34, 0x02, # 0x05 JMP 02H
    0x00,       # 0x07 A1H
    0x03,       # 0x08 22H
    ]
cpu.switches.load_program(rom)

In [ ]:
for _ in range(5):
    cpu.step(instructionwise=True)

In [ ]:
for _ in range(3):
    cpu.step(instructionwise=True, debug=True)

In [ ]:
import ipywidgets as widgets
from IPython.display import display
gui_output_display = widgets.FloatText()
display(gui_output_display)

# monkey patch output function
def display_number(x):
    gui_output_display.value = x

cpu.reg_o.output_function = display_number

In [ ]:
for _ in range(3):
    time.sleep(.2)
    cpu.step(instructionwise=True, debug=False)

In [ ]:
for _ in range(27):
    time.sleep(.03)
    cpu.step(instructionwise=True, debug=False)

In [ ]:
for _ in range(133):
    time.sleep(.03)
    cpu.step(instructionwise=True, debug=False)

# DMA

In [ ]:
import numpy as np
import ipywidgets as widgets
from bokeh.io import push_notebook, show, output_notebook
from bokeh.models import HoverTool
from bokeh.plotting import figure
output_notebook()

In [ ]:
def setup_bokeh_gui(cpu):
    # Zeroed-out memory map for initializing the display
    initial_bitmap = np.zeros((0x10, 0x10))
    
    # store the actual address for use in hovertool
    address_bitmap = np.ndarray((0x10, 0x10), dtype='int')
    for i in range(initial_bitmap.shape[0]):
        for j in range(initial_bitmap.shape[1]):
            address_bitmap[i, j] = i * 0x10 + j
            
    hover = HoverTool(
        tooltips=[("Address", "@address{$%02X}"),
                  ("value", "@image{$%02X}"),
                 ],
        formatters={
            'address' : 'printf',
            'image' : 'printf',
            },
    )

    p = figure(
        title="Bitmap of Memory",
        plot_height=350,
        plot_width=350,
        tools="",
        background_fill_color='#efefef',
        )
    

    p.add_tools(hover)
    p.toolbar_location = None

    i = p.image(image=[initial_bitmap],
                x=-0.5, y=-0.5, dw=0xF+1, dh=0xF+1, 
                palette="Spectral11",
               )
    i.data_source.data['address'] = [address_bitmap]
    show(p, notebook_handle=True)
    
    
    def plt_memory(bitmap):
        """Refresh the bokeh plot with new bitmap"""
        i.data_source.data['image'] = [bitmap]
        push_notebook()
        
    cpu.dma.connect_dma_handler(plt_memory)

In [ ]:
cpu = Computer()

In [ ]:
countup = """
    LDA #$99
    ADD #$0E
    DMA
    STA  $4A
    JMP  $02
    """

countup_bytes = assemble(countup)

In [ ]:
moveacross = """
    LDA #$10
    loop:
    ADD #$03
    DMA
    STA  colorbuffer
    STA (colorbuffer)
    JMP  loop
    colorbuffer:
    """
moveacross_bytes = assemble(moveacross)

In [ ]:
cpu.reset()
cpu.switches.load_program(countup_bytes)
setup_bokeh_gui(cpu)

for i in range(400):
    time.sleep(.01)
    if i % 50 == 0:
        print(i)
    cpu.step(instructionwise=True, debug=False)

In [ ]:
cpu = Computer()
cpu.reset()
cpu.switches.load_program(moveacross_bytes)
setup_bokeh_gui(cpu)

for i in range(400):
    #time.sleep(.01)
    cpu.step(instructionwise=True, debug=False)

# Write your own program

In [ ]:
program = """
    LDA #locationbuffer
    ADD #$01
    STA locationbuffer
    ;BAI
    STA colorbuffer
loop:
    DMA
    LDA  colorbuffer
    ADD #$01
    STA  colorbuffer
    
    LDA locationbuffer
    ADD #$02
    STA locationbuffer
    
    LDA colorbuffer
    STA (locationbuffer)
    JMP  loop
colorbuffer:
    BYTE #33 44 55 66
locationbuffer:
    NOP
"""

program_bytes = assemble(program)

cpu = Computer()
cpu.reset()
cpu.switches.load_program(program_bytes)
setup_bokeh_gui(cpu)

for i in range(999):
    time.sleep(.003)
    cpu.step(instructionwise=True, debug=False)

In [ ]:
import IPython
ipython = IPython.get_ipython()

def setup_ipywidget_input_reg(cpu):
    slider = widgets.IntSlider(value=0x90, min=0x00, max=0xFF, step=1)
    display(slider)
    def update_input_reg():
        ipython.kernel.do_one_iteration()
        return slider.value
    cpu.reg_c.input_function = update_input_reg

In [ ]:
program = """
    ;Start the drawing after the last buffer
start:
    LDA #locationbuffer
    ADD #$01
    STA locationbuffer
loop:
    BAI
    STA (locationbuffer)

    
    DMA
    
    LDA locationbuffer
    ADD #$01
    STA locationbuffer
    
    BNZ  loop
    JMP  start
colorbuffer:
    BYTE #33 44 55 66
locationbuffer:
    NOP
"""

program_bytes = assemble(program)

cpu = Computer()
cpu.reset()
cpu.switches.load_program(program_bytes)
setup_bokeh_gui(cpu)
setup_ipywidget_input_reg(cpu)

for i in range(9819):
    time.sleep(.005)
    cpu.step(instructionwise=True, debug=False)